In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_dir = '/home/lzhao/data/tmp/crypto'

In [3]:
origin_train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
supp_train_df = pd.read_csv(os.path.join(data_dir, 'supplemental_train.csv'))
asset_details_df = pd.read_csv(os.path.join(data_dir, 'asset_details.csv'))

In [4]:
origin_train_df.head(5)

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [5]:
supp_train_df.head(5)

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1623542400,3,1201.0,1.478556,1.48603,1.47800,1.483681,654799.561103,1.481439,-0.002594
1,1623542400,2,1020.0,580.306667,583.89000,579.91000,582.276667,1227.988328,581.697038,-0.009143
2,1623542400,0,626.0,343.789500,345.10800,343.64000,344.598000,1718.832569,344.441729,-0.004525
3,1623542400,1,2888.0,35554.289632,35652.46465,35502.67000,35602.004286,163.811537,35583.469303,0.003096
4,1623542400,4,433.0,0.312167,0.31260,0.31192,0.312208,585577.410442,0.312154,0.001426


In [6]:
train_df = pd.concat([origin_train_df, supp_train_df]).drop_duplicates()

In [7]:
asset_details_df.head(2)

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin


In [5]:
asset_details_df.Asset_ID.unique()

array([ 2,  0,  1,  5,  7,  6,  9, 11, 13, 12,  3,  8, 10,  4])

# 2. Feature Engineering

In [14]:
# Two features from the competition tutorial
def upper_shadow(df): 
    return df['High'] - np.maximum(df['Close'], df['Open'])
    
def lower_shadow(df): 
    return np.minimum(df['Close'], df['Open']) - df['Low']

In [8]:
train_df['date'] = pd.to_datetime(train_df.timestamp, unit='s')
train_df = train_df.sort_values(by='date')

In [9]:
groups, _ = pd.factorize(train_df.date.dt.year.astype(str) + '_'  \
    + train_df.date.dt.month.astype(str) + '_' \
    + train_df.date.dt.day.astype(str))

train_df['groups'] = groups


In [12]:
train_df.head(5)

,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,groups
0,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218,0
1,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399,0
2,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643,0
3,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922,0
4,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264,0


In [11]:
train_df.drop(columns=['timestamp', 'date'], inplace=True)

In [15]:
train_df['upper_Shadow'] = upper_shadow(train_df)
train_df['lower_Shadow'] = lower_shadow(train_df)
train_df["high_div_low"] = train_df["High"] / train_df["Low"]
train_df["open_sub_close"] = train_df["Open"] - train_df["Close"]

In [16]:
# 填充空值 以及 极值
train_df = train_df.fillna(0)
train_df = train_df.replace([np.inf, -np.inf], 0)

In [22]:
train_df.head(5)

,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,groups,upper_Shadow,lower_Shadow,high_div_low,open_sub_close
0,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218,0,22.920,17.4500,1.017971,1.990
1,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399,0,0.000,0.0000,1.000000,0.000
2,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643,0,163.624,169.0840,1.025442,-14.982
3,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922,0,0.000,0.0009,1.000379,0.002
4,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264,0,0.000,0.0030,1.001778,0.043


# 3. Train

In [23]:
feature_names = [i for i in train_df.columns if i not in ['Target', 'date', 'timestamp', 'VWAP', 'Asset_ID', 'groups']]

In [28]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.utils.validation import _deprecate_positional_args
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from lightgbm import LGBMRegressor
import optuna


In [25]:
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [26]:
y_labels = train_df['Target'].values
X_train = train_df[feature_names].values
groups = train_df['groups'].values

In [27]:
cv = PurgedGroupTimeSeriesSplit(
    n_splits = 3,
    max_train_group_size = 50, 
    group_gap = 20, 
    max_test_group_size=60
)

In [29]:
def objective(trial, cv=cv, cv_fold_func=np.average):

    # Optuna suggest params
    param_lgb = {
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }    
    # setup the pieline
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    scaler = StandardScaler()
    
    clf = LGBMRegressor(**param_lgb)

    pipe = Pipeline(steps=[
        ('imputer', imp_mean),
        ('scaler', scaler),
        ('catb', clf)
    ])

    # fit for all folds and return composite MAE score
    maes = []
    for i, (train_idx, valid_idx) in enumerate(cv.split(
        X_train,
        y_labels,
        groups=groups)):
        
        #train_data = X_train[train_idx, :], y_labels[train_idx]
        #valid_data = X_train[valid_idx, :], y_labels[valid_idx]
        
        _ = pipe.fit(X_train[train_idx, :], y_labels[train_idx])
        preds = pipe.predict(X_train[valid_idx, :])
        mae = mean_absolute_error(y_labels[valid_idx], preds)
        maes.append(mae)
    
    print(f'Trial done: mae values on folds: {maes}')
    return -1.0 * cv_fold_func(maes)

In [30]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=60)

[I 2021-11-24 00:36:07,255] A new study created in memory with name: no-name-db08749b-2748-4072-aec2-3133c4fccef3


[LightGBM] [Warning] bagging_fraction is set=0.9303019260823465, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9303019260823465
[LightGBM] [Warning] lambda_l2 is set=1.2861876229819642e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.2861876229819642e-07
[LightGBM] [Warning] lambda_l1 is set=9.009247939993385e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.009247939993385e-08
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.743514710219249, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.743514710219249
[LightGBM] [Warning] bagging_fraction is set=0.9303019260823465, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9303019260823465
[LightGBM] [Warning] lambda_l2 is set=1.2861876229819642e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.2861876229819642e-07
[LightGBM] [Warning] la

[I 2021-11-24 00:36:49,458] Trial 0 finished with value: -0.003037007206310564 and parameters: {'lambda_l1': 9.009247939993385e-08, 'lambda_l2': 1.2861876229819642e-07, 'num_leaves': 207, 'feature_fraction': 0.743514710219249, 'bagging_fraction': 0.9303019260823465, 'bagging_freq': 1, 'min_child_samples': 88}. Best is trial 0 with value: -0.003037007206310564.


Trial done: mae values on folds: [0.004315756655382553, 0.002600596657601222, 0.002194668305947917]
[LightGBM] [Warning] bagging_fraction is set=0.7492805852488086, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7492805852488086
[LightGBM] [Warning] lambda_l2 is set=0.00036533446366190215, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00036533446366190215
[LightGBM] [Warning] lambda_l1 is set=0.05348738972818243, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.05348738972818243
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.5461218651520628, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5461218651520628
[LightGBM] [Warning] bagging_fraction is set=0.7492805852488086, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7492805852488086
[LightGBM] [Warning] lambda_l2 is set=0.00036533446366190215, reg_l

[I 2021-11-24 00:37:18,905] Trial 1 finished with value: -0.0029940090119364303 and parameters: {'lambda_l1': 0.05348738972818243, 'lambda_l2': 0.00036533446366190215, 'num_leaves': 35, 'feature_fraction': 0.5461218651520628, 'bagging_fraction': 0.7492805852488086, 'bagging_freq': 6, 'min_child_samples': 64}. Best is trial 1 with value: -0.0029940090119364303.


...

Trial done: mae values on folds: [0.004177703997003175, 0.002555931782091462, 0.0021580558810964596]
[LightGBM] [Warning] bagging_fraction is set=0.5235950853950513, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5235950853950513
[LightGBM] [Warning] lambda_l2 is set=4.792317835910924, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.792317835910924
[LightGBM] [Warning] lambda_l1 is set=3.647396609497688, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.647396609497688
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.5873713258569121, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5873713258569121
[LightGBM] [Warning] bagging_fraction is set=0.5235950853950513, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5235950853950513
[LightGBM] [Warning] lambda_l2 is set=4.792317835910924, reg_lambda=0.0 will be 

[I 2021-11-24 01:07:30,831] Trial 58 finished with value: -0.0029586189148888033 and parameters: {'lambda_l1': 3.647396609497688, 'lambda_l2': 4.792317835910924, 'num_leaves': 54, 'feature_fraction': 0.5873713258569121, 'bagging_fraction': 0.5235950853950513, 'bagging_freq': 4, 'min_child_samples': 19}. Best is trial 55 with value: -0.002955888566240135.


Trial done: mae values on folds: [0.004167974977108142, 0.0025541361876920887, 0.0021537455798661793]
[LightGBM] [Warning] bagging_fraction is set=0.45828330645395154, subsample=1.0 will be ignored. Current value: bagging_fraction=0.45828330645395154
[LightGBM] [Warning] lambda_l2 is set=0.024294736583156326, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.024294736583156326
[LightGBM] [Warning] lambda_l1 is set=9.998434026401673, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.998434026401673
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.6337887852801547, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6337887852801547
[LightGBM] [Warning] bagging_fraction is set=0.45828330645395154, subsample=1.0 will be ignored. Current value: bagging_fraction=0.45828330645395154
[LightGBM] [Warning] lambda_l2 is set=0.024294736583156326, reg_lambd

[I 2021-11-24 01:07:55,744] Trial 59 finished with value: -0.0029559659347242425 and parameters: {'lambda_l1': 9.998434026401673, 'lambda_l2': 0.024294736583156326, 'num_leaves': 42, 'feature_fraction': 0.6337887852801547, 'bagging_fraction': 0.45828330645395154, 'bagging_freq': 5, 'min_child_samples': 49}. Best is trial 55 with value: -0.002955888566240135.


Trial done: mae values on folds: [0.004164464970495407, 0.0025502098856226884, 0.0021532229480546337]


In [31]:
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

Best trial:
  Value: -0.002955888566240135


In [32]:
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
best_params = trial.params 

  Params: 
    lambda_l1: 8.869735681562146
    lambda_l2: 0.492799071695039
    num_leaves: 36
    feature_fraction: 0.4050052035596727
    bagging_fraction: 0.40354881118115155
    bagging_freq: 5
    min_child_samples: 18


In [34]:
maes = []
oof = np.zeros(len(X_train))
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
scaler = StandardScaler()
for i, (train_idx, valid_idx) in enumerate(cv.split(X_train, y_labels, groups=groups)):

    clf = LGBMRegressor(**best_params)
    pipe = Pipeline(steps=[('imputer', imp_mean), ('scaler', scaler), ('catb', clf)])
    _ = pipe.fit(X_train[train_idx, :], y_labels[train_idx])
    preds = pipe.predict(X_train[valid_idx, :])
    oof[valid_idx] = preds

    mae = mean_absolute_error(y_labels[valid_idx], preds)
    maes.append(mae)

print(f'Score: {np.mean(maes)}')

[LightGBM] [Warning] lambda_l1 is set=8.869735681562146, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.869735681562146
[LightGBM] [Warning] feature_fraction is set=0.4050052035596727, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4050052035596727
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.492799071695039, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.492799071695039
[LightGBM] [Warning] bagging_fraction is set=0.40354881118115155, subsample=1.0 will be ignored. Current value: bagging_fraction=0.40354881118115155
[LightGBM] [Warning] lambda_l1 is set=8.869735681562146, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.869735681562146
[LightGBM] [Warning] feature_fraction is set=0.4050052035596727, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4050052035596727
[LightGBM] [Warning] bagging_freq is set=

In [36]:
!pwd

/home/lzhao/projects


In [37]:
import gresearch_crypto

In [38]:
env = gresearch_crypto.make_env()


In [39]:
iter_test = env.iter_test()

In [40]:
import traceback

In [45]:
all_df_test = []
pipe_prod = pipe
for i, (df_test, df_pred) in enumerate(iter_test):
    for j , row in df_test.iterrows():
        try:            
            row['upper_Shadow'] = upper_shadow(row)
            row['lower_Shadow'] = lower_shadow(row)
            row["high_div_low"] = row["High"] / row["Low"]
            row["open_sub_close"] = row["Open"] - row["Close"]   
            #x_test = get_features(row)
            row = row.fillna(0)
            row = row.replace([np.inf, -np.inf], 0)
            #x_test = fill_nan_inf(x_test)
            y_pred = pipe_prod.predict([row[feature_names].values])            
        except: 
            y_pred = 0.0
            traceback.print_exc()
        df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
    all_df_test.append(df_test)
    env.predict(df_pred)

In [46]:
len(all_df_test)

0